In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [50]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [51]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-02-01 01:40:11--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2026-02-01 01:40:11 (22.5 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [52]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("Text length:", len(text))
print(text[:300])

Text length: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [53]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("Vocab size:", vocab_size)
print(chars)

Vocab size: 65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [54]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

In [55]:
print(stoi)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [56]:
print(itos)

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


In [57]:
def encode(s):
    return [stoi[c] for c in s]

def decode(indices):
    return "".join([itos[i] for i in indices])

In [58]:
print(encode("hello"))
print(decode(encode("hello")))

[46, 43, 50, 50, 53]
hello


In [59]:
x = "hello"
y = "ello?"

In [60]:
block_size = 64   # sequence length
batch_size = 128

In [61]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

In [62]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

In [63]:
class CharLM(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 64)
        self.lstm = nn.LSTM(64, 128, batch_first=True)
        self.fc = nn.Linear(128, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x

In [64]:
model = CharLM().to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, "M parameters")

0.111873 M parameters


In [65]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [66]:
def train_step():
    x, y = get_batch("train")
    logits = model(x)

    loss = criterion(
        logits.view(-1, vocab_size),
        y.view(-1)
    )

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()


In [67]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    losses = []
    for _ in range(50):
        x, y = get_batch("val")
        logits = model(x)
        loss = criterion(
            logits.view(-1, vocab_size),
            y.view(-1)
        )
        losses.append(loss.item())
    model.train()
    return sum(losses) / len(losses)


In [68]:
steps = 10000

for step in range(steps):
    loss = train_step()

    if step % 500 == 0:
        val_loss = estimate_loss()
        print(f"Step {step} | Train loss {loss:.4f} | Val loss {val_loss:.4f}")


Step 0 | Train loss 4.1688 | Val loss 4.1624
Step 500 | Train loss 2.2486 | Val loss 2.2764
Step 1000 | Train loss 2.0157 | Val loss 2.0655
Step 1500 | Train loss 1.9102 | Val loss 1.9632
Step 2000 | Train loss 1.8376 | Val loss 1.9030
Step 2500 | Train loss 1.7837 | Val loss 1.8630
Step 3000 | Train loss 1.6776 | Val loss 1.8297
Step 3500 | Train loss 1.6559 | Val loss 1.7995
Step 4000 | Train loss 1.6510 | Val loss 1.7781
Step 4500 | Train loss 1.6065 | Val loss 1.7650
Step 5000 | Train loss 1.5923 | Val loss 1.7405
Step 5500 | Train loss 1.5400 | Val loss 1.7308
Step 6000 | Train loss 1.5717 | Val loss 1.7279
Step 6500 | Train loss 1.5633 | Val loss 1.7186
Step 7000 | Train loss 1.5600 | Val loss 1.7066
Step 7500 | Train loss 1.5400 | Val loss 1.6989
Step 8000 | Train loss 1.5255 | Val loss 1.6909
Step 8500 | Train loss 1.5177 | Val loss 1.6806
Step 9000 | Train loss 1.4962 | Val loss 1.6798
Step 9500 | Train loss 1.4494 | Val loss 1.6653


In [69]:
@torch.no_grad()
def generate(model, start_char="H", max_new_chars=300):
    model.eval()
    idx = torch.tensor([[stoi[start_char]]], device=device)

    for _ in range(max_new_chars):
        logits = model(idx)
        logits = logits[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        next_idx = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_idx], dim=1)

    return decode(idx[0].tolist())


In [70]:
print(generate(model, "T"))


Then when, and spear to him.

WARWICK:
That to my faimest before, you belong me.
I provost it her love!

QUEEN ENE:
Spirt the duke 'it of wife,
And with the hoaters, fellow for the comfort of away!
And will I will nou would this facemy and injont:
Thy crreate not buckle fordour to you well, sbull not
